In [ ]:
#@title セル①

#Stable Diffusionのインストール
!pip install --upgrade diffusers[torch] transformers

In [ ]:
#@title セル②

from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
from diffusers.models import AutoencoderKL
import torch

#画像生成に使うモデルデータ
model_id = "stablediffusionapi/anything-v5"#@param ["runwayml/stable-diffusion-v1-5", "SG161222/Realistic_Vision_V5.1_noVAE", "stablediffusionapi/anything-v5"] {allow-input: true}
#画像生成に使うVAE
vae = "stabilityai/sd-vae-ft-mse"#@#param {type:"string"}
vae = AutoencoderKL.from_pretrained(vae)

#画像生成に使うスケジューラー
scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")

#パイプラインの作成
pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, vae=vae, custom_pipeline="lpw_stable_diffusion")

pipe = pipe.to("cuda")

In [ ]:
#@title セル③

import datetime
import os
import random

#txt2img出力画像の保存先
!mkdir -p /content/txt2img_output

# ファイル名に使う日付と時刻のフォーマットを定義する
file_format = "%Y%m%d_%H%M%S"
i=0

#ポジティブプロンプト
prompt = "apple"#@param {type:"string"}
#ネガティブプロンプト
n_prompt = "nsfw"#@#param {type:"string"}
#CFG Scale
CFG_scale = 7#@#param {type:"number"}
#ステップ数
Steps = 20#@#param {type:"number"}
#seed値
seed=-1#@param {type:"number"}
if seed is None or seed == -1:
  inputSeed = random.randint(0, 2147483647)
else:
  valueSeed = seed

#生成枚数
num_images = 3#@param {type:"number"}
#出力画像の横幅
width = 512#@param {type:"number"}
#出力画像の高さ
height = 512#@param {type:"number"}
#出力画像を保存するフォルダ
save_path = "/content/txt2img_output"#@#param {type:"string"}

while i < int(num_images):
  #generator
  if seed is None or seed == -1:valueSeed = inputSeed + i
  generator = torch.Generator(device="cuda").manual_seed(valueSeed)

  #画像を生成
  image = pipe(prompt, negative_prompt=n_prompt, width=width, height=height, generator=generator, guidance_scale=CFG_scale, num_inference_steps=Steps).images[0]

  # 現在の日本時間を取得
  jst_now = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
  #出力する画像の名前を生成する
  file_name = (jst_now.strftime(file_format)+ "_" + str(valueSeed))
  image_name = file_name + f".png"

  #画像を保存する
  save_location = os.path.join(save_path, image_name)
  image.save(save_location)
  i = i + 1